In [100]:
from scipy.sparse import csc_array, identity, dia_array
from scipy.sparse.linalg import expm
import numpy as np

In [101]:
dia = np.exp(np.random.rand(2**21))
dia_array((dia,0), shape=(2**21,2**21)).shape


(2097152, 2097152)

In [103]:
I = identity(5)
dia = [-0.1,-0.33,-0.2,99, 198]*3
X = dia_array((dia,0), shape=(15,15)).tocsc()

In [104]:
expm(X).toarray()

/Users/chayaponthunsetkul/Library/Caches/pypoetry/virtualenvs/newbasisimbq-JkixFnlr-py3.10/lib/python3.10/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


array([[9.04837418e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.18923733e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 8.18730753e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.88903032e+42,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+0

pade approximant:

$$
\exp(x) \approx \frac{
1+ (1/2)x + (1/9)x^2 + (1/72)x^3 + (1/1008)x^4 + (1/30240)x^5
}{
1- (1/2)x + (1/9)x^2 - (1/72)x^3 + (1/1008)x^4 - (1/30240)x^5
}
$$

In [118]:
def pade_expm(x):
    numerator = I + 1/2*x + 1/9*x**2 + 1/72 * x**3 + 1/1008 * x**4 + 1/30240 * x**5
    denominator = I - 1/2*x + 1/9*x**2 - 1/72 * x**3 + 1/1008 * x**4 - 1/30240 * x**5
    return numerator/denominator
def taylor_expm(x):
    return I + I + I/2 + I/6 + I/24
def dia_expm(x: dia_array, scalar: complex):
    shape = x.shape
    dia = scalar*x.diagonal()
    dia = np.exp(dia)
    return dia_array((dia,0), shape=shape)


In [119]:
taylor_expm(X).toarray()

array([[2.70833333, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 2.70833333, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 2.70833333, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 2.70833333, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 2.70833333]])

In [120]:
from time import time

In [121]:
iter = 1000
start = time()
for _ in range(iter):
    expm(I)
print(time() - start)
start = time()
for _ in range(iter):
    dia_expm(I,1)
print(time() - start)


/Users/chayaponthunsetkul/Library/Caches/pypoetry/virtualenvs/newbasisimbq-JkixFnlr-py3.10/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/Users/chayaponthunsetkul/Library/Caches/pypoetry/virtualenvs/newbasisimbq-JkixFnlr-py3.10/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


2.156798839569092
0.01263284683227539


In [122]:
gamma = 3
(dia_expm(I, -1j*gamma) == expm(-1j * gamma *I)).toarray()

/var/folders/qx/csm2q07x4x5dp2z8zh1xdjsm0000gn/T/ipykernel_53823/1970631530.py:2: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  (dia_expm(I, -1j*gamma) == expm(-1j * gamma *I)).toarray()


array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])